In [1]:
import sys,time

string = """ S how
 C inemas
 I n
 M y
 P lace
 L isting
 E verything"""

typing_speed = 390

def slow_type(t):
    for l in t:
        sys.stdout.write(l)
        sys.stdout.flush()
        time.sleep(10.0/typing_speed)
    print('\n')

print('')
slow_type(string)


 S how
 C inemas
 I n
 M y
 P lace
 L isting
 E verything



In [1]:
import requests
import sys
import emoji
from datetime import datetime as dt
from datetime import timedelta
        
def cinemoji(cinema, place):
    d_cinema = get_dict(cinema)
    
    name = d_cinema['name']
    emoticon = emoji.emojize(d_cinema['emoji'])
    
    if place == 'before':
        return emoticon +' '+ name
    elif place == 'after':
        return name +' '+ emoticon
    elif place == 'both':
        return emoticon +' '+ name +' '+ emoticon

In [3]:
print(emoji.emojize(":popcorn:"), 'WOKI', emoji.emojize(":popcorn:"))

print(emoji.emojize(":bread:"), 'Brotfabrik', emoji.emojize(":bread:"))

print(emoji.emojize(":T-Rex:"), 'Rex', emoji.emojize(":T-Rex:"))

print(emoji.emojize(":NEW_button:"), 'Neue Filmbühne', emoji.emojize(":NEW_button:"))

🍿 WOKI 🍿
🍞 Brotfabrik 🍞
🦖 Rex 🦖
🆕 Neue Filmbühne 🆕


In [7]:
movietitle = []
l = ['woki', 'filmbuehne', 'rex', 'brot']
for i in l:
    d = get_dict(i)
    for t in d['title']:
        if t.lower() not in movietitle and 'sneak preview' not in t.lower():
            movietitle.append(t.lower())

for i in movietitle:
    print(i)

killers of the flower moon
wochenendrebellen
one for the road
der exorzist: bekenntnis
sara mardini - gegen den strom
she said
the nun 2
philipp mickenbecker - real life
barbie
oppenheimer
equalizer 3 - the final chapter
trolls - gemeinsam stark
paw patrol: der mighty kinofilm
checker tobi und die reise zu den fliegenden flüssen
elemental
asterix & obelix im reich der mitte
miraculous: ladybug & cat noir - der film
kuddelmuddel bei pettersson und findus
der super mario bros. film
heaven can wait - wir leben jetzt
the lost king
ingeborg bachmann - reise in die wüste
fallende blätter
die unwahrscheinliche pilgerreise des harold fry
die theorie von allem
neue geschichten vom pumuckl
die einfachen dinge
weißt du noch?
anselm - das rauschen der zeit
ein ganzes leben
roh 2023/24: don quixote
exhibition on screen: klimt & the kiss
callas - paris, 1958
fearless flyers
la dolce vita
tori und lokita
beata te – der erzengel und ich
il bambino nascosto – das versteckte kind
mamma roma
margini – am

In [191]:
print('\033[38;5;39mhello\033[0;0m')
print('this')
if type(dt.today()) == dt:
    print('hi')
print(dt.today(), dt.now())

print(return_color('this', '160'))

hello
this
hi
2024-02-24 05:31:16.706275 2024-02-24 05:31:16.706278
this


In [ ]:
import sys, time, random
import emoji
from datetime import datetime as dt
from datetime import timedelta
from shutil import get_terminal_size
import scimple_functions as SF

def slow_type(t):
    typing_speed = 375
    for l in t:
        sys.stdout.write(l)
        sys.stdout.flush()
        time.sleep(10.0/typing_speed)
    print('\n')

def opening():
    string = """     S how
     C inemas
     I n
     M y
     P lace
     L isting
     E verything"""

    print('')
    slow_type(string)
    
opening()

def inputexit(var):
    var = input(var)
    if var == 'exit':
        print('exited session.')
        sys.exit()
    else:
        return var


print(emoji.emojize(":T-Rex:"), 'Rex', emoji.emojize(":T-Rex:"))
print(emoji.emojize(":popcorn:"), 'WOKI', emoji.emojize(":popcorn:"))
print(emoji.emojize(":bread:"), 'Brotfabrik', emoji.emojize(":bread:"))
print(emoji.emojize(":NEW_button:"), 'Neue Filmbühne', emoji.emojize(":NEW_button:"))
print('')

imported = []

print('default is all, sort with arg [t]ime, [a]lphabet, [c]inema, [r]oom, [s]eat\n')

def routine():
    sorter_key = None
    
    while True:
        is_date = False

        p_date = inputexit('date: ').lower().replace(' ', '')

        if '--' in p_date:
            p_date, sorter_key = p_date.split('--', 1)


        if str(p_date).lower() in [l.lower() for l in SF.l_today]:
            p_date = dt.today()

        elif str(p_date).lower() in [l.lower() for l in SF.l_tomorrow]:
            p_date = dt.today() + timedelta(days=1)

        elif str(p_date).lower() in [l.lower() for l in SF.l_dayname]:
            day_hit = SF.l_dayname.index(p_date.capitalize()) %7
            today = dt.today().strftime('')

            if day_hit < dt.today().weekday():
                day_skip = 7 - dt.today().weekday() + day_hit
            elif day_hit == dt.today().weekday():
                day_skip = 7
            else:
                day_skip = day_hit - dt.today().weekday()

            p_date = dt.today() + timedelta(days=day_skip)


        if p_date == '':
            p_date = 'all'
            is_date = True

        else:
            p_date, is_date = SF.detect_date(p_date)

        if is_date:
            break
        else:
            print('invalid date')


    if p_date == 'all':
        p_timestamp = False

    else:
        print('')
        while True:
            p_time = inputexit('time: ').lower().replace(' ', '')

            if p_time == '':
                p_time = '0000'

            if ':' in p_time:
                hour, minute = p_time.split(':', 1)
            else:
                hour = p_time[:-2]
                minute = p_time[-2:]

            try:
                p_timestamp = p_date.replace(hour=int(hour), minute=int(minute), second=0, microsecond=0)
                break

            except ValueError:
                print('invalid time')            

    print('')
    while True:
        p_cinema = inputexit('cinema: ').lower().replace(' ', '').split(',')

        if p_cinema == ['']:
            p_cinema = []
            for i in SF.cinema_alias:
                p_cinema.append(i[0])

        else:
            p_cinema = list(filter(None, p_cinema))

        p_cinema = list(set([SF.is_cinema_alias(p) for p in p_cinema]))

        if False in p_cinema:
            print('invalid cinema key.')
            continue

        else:
            for p in p_cinema:
                if not SF.cinema_in_dlist(p, imported):
                    desc = 'importing ' + p + '...'
                    end = p + ' done.'
                    loader = SF.Loader(desc, end).start()

                    imported.append(SF.import_cinema_errorless(p))

                    loader.stop()
            break    

    p_title = inputexit('\ntitle: ').lower().rstrip()
    if p_title == '':
        p_title = False
        
    print('')
    
    selected_cinema = [event for cinema in imported for selected in p_cinema if selected in cinema['alias'] for event in cinema['event']]
        
    SF.show(selected_cinema, sorter_key, p_timestamp, p_title)
    

while True:
    routine()
    print('\n\n\n\n')


In [6]:
import emoji
s = emoji.emojize(":T-Rex:")
print(len(s))

1


In [8]:
while True:
    p_time = inputexit('time: ').lower().replace(' ', '')

    if p_time == '':
        p_time = '0000'

    if ':' in p_time:
        hour, minute = p_time.split(':', 1)
    else:
        hour = p_time[:-2]
        minute = p_time[-2:]
        
    print(hour, minute)
    print(dt.today().replace(hour=int(hour), minute=int(minute), second=0, microsecond=0).strftime('%H%M'))

time:  2030


20 30
2030


time:  exit


exited session.


SystemExit: 

In [252]:
while True:
    var_cinema = None
    sorter_key = None
    var = dt.today()
    selected_cinema = []
    
    p_date = inputexit('date: ').lower()
    
    if '--' in p_date:
        p_date, flag = p_date.split('--')

    l_cinema = [i for i in l_cinema if type(i) == dict]
    
    date = detect_date(var)
    

    if str(var).lower() in [l.lower() for l in l_today]:
        show_date(dt.today(), selected_cinema, sorter_key)

    elif str(var).lower() in [l.lower() for l in l_tomorrow]:
        show_date(dt.today() + timedelta(days=1), selected_cinema, sorter_key)

    elif str(var).lower() in [l.lower() for l in l_dayname]:
        day_hit = l_dayname.index(var.capitalize())%7
        today = dt.today().strftime('')

        if day_hit < dt.today().weekday():
            day_skip = 7 - dt.today().weekday() + day_hit
        elif day_hit == dt.today().weekday():
            day_skip = 7
        else:
            day_skip = day_hit - dt.today().weekday()

        show_date(dt.today() + timedelta(days = day_skip), selected_cinema, sorter_key)

    elif type(date) == dt:
        show_date(date, selected_cinema, sorter_key)
        
    elif var_cinema != None:
        for i in range(14):
            show_date(dt.today() + timedelta(days = i), selected_cinema, sorter_key)

    else:
        for cinema in selected_cinema:
            for event in cinema['event']:
                if var == event['title']:
                    show_date(date, selected_cinema, sorter_key)
        
    print('\n\n')

prompt:  t..fb,,brot --a



🆕 ALL OF US STRANGERS
------------------------------------------------
24.02 Sat, 18:00, 105min
Neue Filmbühne
https://www.kinoheld.de/kino/bonn/neue-filmbuehne-bonn/vorstellung/2963539253?mode=widget&layout=movies&rb=0&hideFilters=1#panel-seats

🆕 EINE MILLION MINUTEN
------------------------------------------------
24.02 Sat, 20:15, 125min
Neue Filmbühne
https://www.kinoheld.de/kino/bonn/neue-filmbuehne-bonn/vorstellung/2963539088?mode=widget&layout=movies&rb=0&hideFilters=1#panel-seats

🆕 RAUS AUS DEM TEICH
------------------------------------------------
24.02 Sat, 13:45, 95min
Neue Filmbühne
https://www.kinoheld.de/kino/bonn/neue-filmbuehne-bonn/vorstellung/2963539388?mode=widget&layout=movies&rb=0&hideFilters=1#panel-seats

🆕 THE HOLDOVERS
------------------------------------------------
24.02 Sat, 15:30, 135min
Neue Filmbühne
https://www.kinoheld.de/kino/bonn/neue-filmbuehne-bonn/vorstellung/2963539298?mode=widget&layout=movies&rb=0&hideFilters=1#panel-seats

🍞 Colonos
--------

prompt:  exit


exited session.


SystemExit: 

In [ ]:

    if '--' in prompt:
        prompt, sorter_key = prompt.split('--')
    
    if prompt[-1] == ' ':
        prompt = prompt[:-1]
        
    print('')
    
    if '..' in prompt:
        var, var_cinema = prompt.split('..', 1)
        var_cinema = multicinema(var_cinema)
    
    elif is_cinema_alias(prompt):
        var_cinema = multicinema(prompt)

    else:
        var = prompt
    
    if type(var_cinema) == list:
        for cinema in var_cinema:
            if not cinema_in_dlist(cinema, l_cinema):
                l_cinema.append(import_cinema(cinema))
            
            for d_cinema in l_cinema:
                if cinema == d_cinema['alias'][0]:
                    selected_cinema.append(d_cinema)
            
    elif var_cinema == None:
        if l_cinema:
            for name in [c[0] for c in cinema_alias]:
                if name not in [c['alias'][0] for c in l_cinema]:
                    l_cinema.append(import_cinema(name))
        else:
            for name in [c[0] for c in cinema_alias]:
                l_cinema.append(import_cinema(name))
                
        selected_cinema = l_cinema
    
    elif var_cinema not in l_daybuzz and not detect_date(var_cinema):
        print('This cinema', var_cinema[l_cinema.index(cinema)], 'is unknown.')
        continue

In [7]:
woki = wk.woki()

In [12]:
brot = bf.brotfabrik()

In [18]:
rex = rx.rex()

In [19]:
buehne = fb.filmbuehne()

In [22]:
l_cinema = (woki, brot, rex, buehne)

In [16]:
for i in woki:
    if i['timestamp'].strftime('%d.%m') == dt.today().strftime('%d.%m'):
        show_event(i)

In [19]:
def show_cinema(d_cinema, place):
    print(cinemoji(d_cinema['short name'], place))
    
    for i in range(len(d_cinema['title'])):
        print(d_cinema['title'][i])
        
        if d_cinema['runtime'][i] != None:
            d_cinema['runtime'][i] = str(d_cinema['runtime'][i]) + ' min'
            
        print(d_cinema['timestamp'][i].strftime('%d.%m. %a, %H:%M;'), 'runtime:', d_cinema['runtime'][i])
        
        place = d_cinema['location'][i]
        if d_cinema['room'][i] != None:
            place = place +': room '+ d_cinema['room'][i]
        print(place)
        
        print(f"\033[38;5;26m{d_cinema['ticket'][i]}\x1b[0m")
        
        print('\n')

before = time.time()
d_cinema = get_dict('rex')
print(time.time()-before)

show_cinema(d_cinema, 'both')

AttributeError: 'str' object has no attribute 'time'

In [21]:
from urllib.request import urlopen
from urllib.parse import unquote
import re
import json
from datetime import datetime as dt


timestamp = []
title = []
room = []
spec = []
location = []
ticket = []

url = 'https://www.imdb.com/title/tt11198810/'
page = urlopen(url)
html = page.read().decode("utf-8")

soup = BeautifulSoup(html, "html.parser")
print(soup)

#match_results = 
#jstring = json.loads(match_results)
#print(json.dumps(jstring, indent=3))
#pattern = r'<div class="relative">.*?</div>'
#match_results = re.findall(pattern, html, re.DOTALL)

HTTPError: HTTP Error 403: Forbidden

In [63]:
movie = ia.search_movie('harry potter')[0]
print(movie)
movie.infoset2keys

Harry Potter and the Sorcerer's Stone


{}

In [16]:
import time
from imdb import Cinemagoer

ia = Cinemagoer()

the_matrix = ia.get_movie('0133093', info=['main'])
print('rating is', the_matrix.get('rating'))

rating is 8.7


In [14]:
import time
ia = Cinemagoer()

for i in movietitle[6:10]:
    print('##', i)

    before = time.time()
    movie = ia.search_movie(i)[0].movieID
    print(movie)
    print(time.time()-before)
    before = time.time()
    movies = ia.get_movie(movie, info=['rating'])
    #movie = ia.get_movie(movie, info=['cast'])
    #print(movie.infoset2keys)
    #print(ia.get_movie(movie, info=['rating']))
    print(movies.get('rating'))
    movies = ia.get_movie(movie)
    print(movies.infoset2keys)
    print(time.time()-before)
    before = time.time()


## the nun 2


2023-10-25 01:48:24,913 ERROR [imdbpy] /Users/tarowatanabe/opt/anaconda3/lib/python3.8/site-packages/imdb/__init__.py:844: unknown information set "rating"


10160976
1.3201336860656738
None
{'main': ['localized title', 'cast', 'genres', 'runtimes', 'countries', 'country codes', 'language codes', 'color info', 'aspect ratio', 'box office', 'certificates', 'original air date', 'rating', 'votes', 'cover url', 'imdbID', 'videos', 'plot outline', 'languages', 'title', 'year', 'kind', 'original title', 'director', 'writer', 'producer', 'composer', 'cinematographer', 'editor', 'editorial department', 'casting director', 'production design', 'art direction', 'set decoration', 'costume designer', 'make up', 'production manager', 'assistant director', 'art department', 'sound crew', 'special effects', 'visual effects', 'stunt performer', 'camera and electrical department', 'casting department', 'costume department', 'location management', 'music department', 'script department', 'transportation department', 'miscellaneous crew', 'akas', 'production companies', 'distributors', 'special effects companies', 'other companies'], 'plot': ['plot', 'synopsi

2023-10-25 01:48:28,576 ERROR [imdbpy] /Users/tarowatanabe/opt/anaconda3/lib/python3.8/site-packages/imdb/__init__.py:844: unknown information set "rating"


28627002
0.8141398429870605
None
{'main': ['localized title', 'genres', 'runtimes', 'countries', 'country codes', 'language codes', 'color info', 'certificates', 'rating', 'votes', 'cover url', 'imdbID', 'plot outline', 'languages', 'title', 'year', 'kind', 'original title', 'director', 'writer', 'producer', 'composer', 'cinematographer', 'editor', 'production companies'], 'plot': ['plot']}
2.3289988040924072
## barbie


2023-10-25 01:48:31,888 ERROR [imdbpy] /Users/tarowatanabe/opt/anaconda3/lib/python3.8/site-packages/imdb/__init__.py:844: unknown information set "rating"


1517268
0.9829950332641602
None
{'main': ['localized title', 'cast', 'genres', 'runtimes', 'countries', 'country codes', 'language codes', 'color info', 'aspect ratio', 'sound mix', 'box office', 'certificates', 'original air date', 'rating', 'votes', 'cover url', 'imdbID', 'videos', 'languages', 'title', 'year', 'kind', 'original title', 'director', 'writer', 'producer', 'composer', 'cinematographer', 'editor', 'editorial department', 'casting director', 'production design', 'art direction', 'set decoration', 'costume designer', 'make up', 'production manager', 'assistant director', 'art department', 'sound crew', 'special effects', 'visual effects', 'stunt performer', 'camera and electrical department', 'animation department', 'casting department', 'costume department', 'location management', 'music department', 'script department', 'transportation department', 'miscellaneous crew', 'thanks', 'akas', 'production companies', 'distributors', 'special effects companies', 'other companie

2023-10-25 01:48:36,050 ERROR [imdbpy] /Users/tarowatanabe/opt/anaconda3/lib/python3.8/site-packages/imdb/__init__.py:844: unknown information set "rating"


15398776
1.0211632251739502
None
{'main': ['localized title', 'cast', 'genres', 'runtimes', 'countries', 'country codes', 'language codes', 'color info', 'aspect ratio', 'sound mix', 'box office', 'certificates', 'original air date', 'rating', 'votes', 'cover url', 'imdbID', 'videos', 'plot outline', 'languages', 'title', 'year', 'kind', 'original title', 'director', 'writer', 'producer', 'composer', 'cinematographer', 'editor', 'editorial department', 'casting director', 'production design', 'art direction', 'set decoration', 'costume designer', 'make up', 'production manager', 'assistant director', 'art department', 'sound crew', 'special effects', 'visual effects', 'stunt performer', 'camera and electrical department', 'animation department', 'casting department', 'costume department', 'location management', 'music department', 'script department', 'transportation department', 'miscellaneous crew', 'thanks', 'akas', 'top 250 rank', 'production companies', 'distributors', 'special ef

In [ ]:
import time
ia = Cinemagoer()
print(ia.get_movie_infoset())

for i in movietitle:
    print('##', i)

    before = time.time()
    movie = ia.search_movie(i)[0]
    print(movie)
    print('found')
    print(time.time()-before)
    before = time.time()

    ia.update(movie, info=['rating'])
    print(time.time()-before)
    before = time.time()

    print('info', movie.current_info)
    print(time.time()-before)
    before = time.time()

    print(type(movie))

    print(time.time()-before)
    before = time.time()

    try:
        print(movie.infoset2keys)
    except KeyError:
        print('this')
    print('\n')

In [44]:
try:
    print(movie.infoset2keys)
except KeyError:
    print('this')
print('\n')

{}




In [35]:
brot = bf.brotfabrik()

ia = Cinemagoer()
print(ia.get_movie_infoset())

print(resp)

for i in movietitle:
    try:
        print('##', i)
        
        movie = ia.search_movie(i)[0]
        for key, value in movie.items():
            print(key, value)
        print('\n')
        
        rtmovie = rt.Movie(i)
        if movie['year'] == rt.year_released(i):
            print(rtmovie, '\n')
        else:
            raise KeyError
        
    except KeyError:
        try:
            ia.update(movie, info=['main'][0])
            print(movie.get('original title'))
            movie = rt.Movie(movie.get('original title'))
            print(movie, '\n')
        except:
            print('no ratings on tomatoes')
            continue
        continue

['airing', 'akas', 'alternate versions', 'awards', 'connections', 'crazy credits', 'critic reviews', 'episodes', 'external reviews', 'external sites', 'faqs', 'full credits', 'goofs', 'keywords', 'list', 'locations', 'main', 'misc sites', 'news', 'official sites', 'parents guide', 'photo sites', 'plot', 'quotes', 'recommendations', 'release dates', 'release info', 'reviews', 'sound clips', 'soundtrack', 'synopsis', 'taglines', 'technical', 'trivia', 'tv schedule', 'video clips', 'vote details']


NameError: name 'resp' is not defined

In [171]:
brot = bf.brotfabrik()

ia = Cinemagoer()
print(ia.get_movie_infoset())

for i in brot['title']:
    print(i)
    movie = ia.search_movie(i)
    for m in movie:
        print(m['title'], type(m))

        for key, value in m.items() :
            print (key, value)

        print('\n')

NameError: name 'bf' is not defined

In [ ]:
ia = Cinemagoer()
print(ia.get_movie_infoset())

movie = ia.search_movie('rose')

print(movie, type(movie[2]))
for m in movie:
    print(m['title'], type(m))
    
    for key, value in m.items() :
        print (key, value)
        
    print('\n')

In [5]:
# create an instance of the Cinemagoer class
ia = Cinemagoer()

# get a movie
movie = ia.get_movie('0133093')

# print the names of the directors of the movie
print('Directors:')
for director in movie['directors']:
    print(director['name'])

# print the genres of the movie
print('Genres:')
for genre in movie['genres']:
    print(genre)

# search for a person name
people = ia.search_person('Mel Gibson')
for person in people:
    print(person.personID, person['name'])

Directors:
Lana Wachowski
Lilly Wachowski
Genres:
Action
Sci-Fi
0000154 Mel Gibson
12550288 Mel Gibson
0317112 Mel Gibson
2153167 Mel Gibson
15271356 Mel Gibson
4798507 Mel A. Gibson
3323885 Adam Taylor
2800055 Mel Gibson
3742421 Melina Mari Bryant
0317096 Meg Gibson
0879085 Tyrese Gibson
0684545 Julie Pinson
8908771 Millie Gibson
0316995 Hoot Gibson
0316945 Donal Gibson
4047961 Noel Gibson
6521923 Mel Gibbon
2988370 Mal Gibson
0316932 Dean Paul Gibson
0316878 Cal Gibson


In [36]:
for i in movietitle:
    print('-'*48)
    movieinput = i

    resp = requests.get('https://rotten-tomatoes-api.ue.r.appspot.com/search/' + movieinput)

    print(resp.json(), '\n')
    print(len(resp.json().get('movies')))

    i = 0
    if len(resp.json().get('movies')) > 1:
        for m in resp.json().get('movies'):
            i += 1
            print(f'{i:2.0f}.', m.get('name'))
            print(' '*3, 'release:', m.get('year'))

        hit = int(abort(input('-> select wich file: ')))
        print(resp.json().get('movies')[hit - 1])

    else:
        print(resp.json().get('movies')[0].get('name'))


------------------------------------------------
{'movies': [{'name': 'Killers of the Flower Moon', 'tomatometer': 92, 'audience_score': 85, 'weighted_score': 89, 'genres': ['Crime', 'Drama'], 'rating': 'R', 'duration': '3h 26m', 'year': '2023', 'actors': ['Leonardo DiCaprio', 'Robert De Niro', 'Jesse Plemons', 'Lily Gladstone', 'Tantoo Cardinal'], 'directors': ['Martin Scorsese']}]} 

1
Killers of the Flower Moon
------------------------------------------------
{'movies': []} 

0


IndexError: list index out of range

In [ ]:
from urllib.request import urlopen
from urllib.parse import unquote
import re
import json
from datetime import datetime as dt


timestamp = []
title = []
room = []
spec = []
location = []
ticket = []

url = 
page = urlopen(url)
html = page.read().decode("utf-8")

soup = BeautifulSoup(html, "html.parser")
print(soup)

#match_results = 
#jstring = json.loads(match_results)
#print(json.dumps(jstring, indent=3))
#pattern = r'<div class="relative">.*?</div>'
#match_results = re.findall(pattern, html, re.DOTALL)

# Kinopolis

In [153]:
from urllib.request import urlopen
from urllib.parse import unquote
import re
import json
from datetime import datetime as dt


timestamp = []
title = []
room = []
spec = []
location = []
ticket = []

url = "https://www.kinopolis.de/bn/vorverkauf"
page = urlopen(url)
html = page.read().decode("utf-8")

pattern = r'<section class="bg.*?</section>'
match_results = re.findall(pattern, html, re.DOTALL)

for section in match_results[:-1]:
    program = BeautifulSoup(section, "html.parser")   
    
    pattern = r'<h2 class="hl--1 hidden-max-xs">.*?</h2>'
    version = re.findall(pattern, str(program), re.DOTALL)
    _, _, name, rest = str(version).split('>', 3)
    jtitle, _ = name.split('<', 1)
#    _, rest = rest.split('>', 1)
#    if len(rest) > 2:
#        jtitle2, _ = rest.split('<', 1)
        
    pattern = r"datetime.*?</time>"
    jtime = re.findall(pattern, section, re.DOTALL)
    i = 0    
    
    #print(program.prettify())    
    for prog in program.find_all("div", {"class": "prog"}):
        for day in prog.find_all("a", {"class": "prog__day"}):
            day = str(day)
            
            #if jtitle2:
            #    title.append(jtitle, jtitle2)
            #else:
            #    title.append(jtitle)
            
            title.append(jtitle)
            
            pattern = r"data-version='.*?'"
            jspec = ','.join(re.findall(pattern, day, re.DOTALL))
            jspec = jspec.replace('data-version=', '').replace('"', '')
            jspec = jspec.replace('[', '').replace("'", '').replace("]", '')
            

            pattern = r'href=".*?"'
            ticket.append('https://www.kinopolis.de' + re.findall(pattern, day, re.DOTALL)[0][6:-1])

            label = [r'prog__label.*?</div>',
                    r'prog__time.*?</div>',
                    r'prog__version.*?</div>',
                    r'prog__extras.*?</div>',
                    r'prog__cinema.*?</div>']

            for pattern in label:
                _, label = re.findall(pattern, day, re.DOTALL)[0].split('>', 1)
                label, _ = label.split('<', 1)
                if 'time' in pattern:
                    timestamp.append(dt.strptime(jtime[i][10:-21] +'.'+ label, '%Y-%m-%d.%H:%M')) 
                elif 'cinema' in pattern:
                    room.append(label.replace('Kino', '').replace(' ', ''))
                elif label:
                    jspec += ', ' + label
                    
            spec.append(jspec)                    
            location.append('KINOPOLIS')
                    
        i += 1
        

for i in range(len(title)):
    print(timestamp[i])
    print(title[i])
    print(ticket[i])
    print(spec[i])
    print(location[i], "\n")

print('done')

done


In [154]:
from urllib.request import urlopen
from urllib.parse import unquote
import re
import json
from datetime import datetime as dt


timestamp = []
title = []
room = []
spec = []
location = []
ticket = []

url = "https://www.kinopolis.de/bn/programm/woche-1"
page = urlopen(url)
html = page.read().decode("utf-8")

pattern = r'<section class="bg.*?</section>'
match_results = re.findall(pattern, html, re.DOTALL)

for section in match_results[:-1]:        
    program = BeautifulSoup(section, "html.parser")   
    
    pattern = r'<h2 class="hl--1 hidden-max-xs">.*?</h2>'
    version = re.findall(pattern, str(program), re.DOTALL)
    _, _, name, rest = str(version).split('>', 3)
    jtitle, _ = name.split('<', 1)
#    _, rest = rest.split('>', 1)
#    if len(rest) > 2:
#        jtitle2, _ = rest.split('<', 1)
        
    pattern = r"datetime.*?</time>"
    jtime = re.findall(pattern, section, re.DOTALL)
    i = 0    
    
    #print(program.prettify())    
    for prog in program.find_all("div", {"class": "prog"}):        
        for day in prog.find_all("a", {"class": "prog__day"}):
            day = str(day)
            
            #if jtitle2:
            #    title.append(jtitle, jtitle2)
            #else:
            #    title.append(jtitle)
            
            title.append(jtitle)
            
            pattern = r"data-version='.*?'"
            jspec = ','.join(re.findall(pattern, day, re.DOTALL))
            jspec = jspec.replace('data-version=', '').replace('"', '')
            jspec = jspec.replace('[', '').replace("'", '').replace("]", '')
            

            pattern = r'href=".*?"'
            ticket.append('https://www.kinopolis.de' + re.findall(pattern, day, re.DOTALL)[0][6:-1])

            label = [r'prog__label.*?</div>',
                    r'prog__time.*?</div>',
                    r'prog__version.*?</div>',
                    r'prog__extras.*?</div>',
                    r'prog__cinema.*?</div>']

            for pattern in label:
                _, label = re.findall(pattern, day, re.DOTALL)[0].split('>', 1)
                label, _ = label.split('<', 1)
                if 'time' in pattern:
                    timestamp.append(dt.strptime(jtime[i][10:-21] +'.'+ label, '%Y-%m-%d.%H:%M')) 
                elif 'cinema' in pattern:
                    room.append(label.replace('Kino', '').replace(' ', ''))
                elif label:
                    jspec += ', ' + label
                    
            spec.append(jspec)                    
            location.append('KINOPOLIS')
                    
        i += 1
        
for i in range(len(title)):
    print(timestamp[i])
    print(title[i])
    print(ticket[i])
    print(spec[i])
    print(location[i], "\n")

print('done')

[]
done
